# Submission 2. Fast-ai

| Variable | Definition                      | Key                                         |
|----------|---------------------------------|---------------------------------------------|
| survival | Survival                        | 0 = No, 1 = Yes                             |
| pclass   | Ticket class                    | 1 = 1st, 2 = 2nd, 3 = 3rd                   |
| sex      | Sex                             |                                             |
| Age      | Age in years                    |                                             |
| sibsp    | # of siblings / spouses aboard the Titanic |                               |
| parch    | # of parents / children aboard the Titanic  |                               |
| ticket   | Ticket number                   |                                             |
| fare     | Passenger fare                  |                                             |
| cabin    | Cabin number                    |                                             |
| embarked | Port of Embarkation             | C = Cherbourg, Q = Queenstown, S = Southampton |

In [1]:
import pandas as pd
from pathlib import Path
import os
from fastai.tabular.all import *
from sklearn.model_selection import StratifiedKFold

In [2]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/titanic')
    !pip install -Uqq fastai
else:
    import zipfile,kaggle
    path = Path('titanic')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [3]:
pd.options.display.float_format = '{:.2f}'.format
set_seed(42)

In [4]:
# Read train and test files into pandas dataframes
df = pd.read_csv('train.csv')
df_submission = pd.read_csv('test.csv')

# Preprocessing


Feature Engineering

In [5]:
def add_features(df):
    df['LogFare'] = np.log1p(df['Fare'])
    df['Deck'] = df.Cabin.str[0].map(dict(A="ABC",B="ABC", C="ABC", D="DE", E="DE", F="FG", G="FG"))
    df['Family'] = df.SibSp+df.Parch
    df['Alone'] = df.Family==0
    df['TicketFreq'] = df.groupby('Ticket')['Ticket'].transform('count')
    df['Title'] = df.Name.str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    df['Title'] = df.Title.map(dict(Mr="Mr",Miss="Miss",Mrs="Mrs",Master="Master"))

add_features(df)

# Modeling with Fast-ai and StratifiedKFold from Sk-learn

In [6]:
# Create splits
splits = RandomSplitter(seed=42)(df)

In [7]:

def run_fold(train_idx, valid_idx):
    # val_pct = []
    # tst_preds = []

    # Create DataLoader
    dls = TabularPandas(df,        
        procs = [Categorify, FillMissing, Normalize],
        cat_names=["Sex","Pclass","Embarked","Deck", "Title"],
        cont_names=['Age', 'SibSp', 'Parch', 'LogFare','Alone', 'TicketFreq', 'Family'],
        y_names="Survived", y_block = CategoryBlock(),
        splits=(list(train_idx),list(valid_idx))
    ).dataloaders(path=".")

    # Create learner. Size of each hidden layer ([100,50])
    learn = tabular_learner(dls, layers=[10,10], metrics=accuracy)

    # Train model
    learn.fit_one_cycle(5) # Adjust epochs

    # val_pct.append(learn.validate()[1])

    # a,b = learn.get_preds(ds_idx=2)
    # tst_preds.append(a)

    
    # Evaluate model - you can return validation loss, accuracy or any other metric
    return learn.validate()[1]#,val_pct, tst_preds

In [ ]:
# Prepare StratifiedKFold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Run K-Fold cross-validation
scores = [run_fold(train_idx, valid_idx) for train_idx, valid_idx in kf.split(df)]

In [ ]:
# Calculate average score
average_score = sum(scores) / len(scores)
print(f'Average Accuracy: {average_score}')

# Submit to Kaggle

Prepare the data

In [ ]:
df_submission = pd.read_csv(path/'test.csv') # Read test file
df_submission['Fare'] = df_submission.Fare.fillna(0)# There is one Fare missing
add_features(df_submission) # Add features to test dataframe
dl_test = learn.dls.test_dl(df_submission) # Create DataLoader for test dataframe from the configurations of the learner

Get predictions

In [ ]:
preds,_=learn.get_preds(dl=dl_test) # Get predictions

Create submission file

In [ ]:
df_submission['Survived'] = (preds[:,1]>0.5).int() # Add predictions to dataframe
sub_df_submission = df_submission[['PassengerId','Survived']] # Create submission dataframe
sub_df_submission.to_csv('sub2.csv', index=False) # Save submission dataframe to csv file

In [ ]:
!head sub.csv

In [ ]:
!kaggle competitions submit -c titanic -f sub2.csv -m "Fastai Tabular Learner with 10,10 layers. 16 epochs. 0.03 learning rate. 0.5 threshold."